In [15]:
import pandas as pd
from time import time
from pydeseq2.dds import DeseqDataSet
from pydeseq2.ds import DeseqStats

ModuleNotFoundError: No module named 'pydeseq2'

In [7]:
def preprocess_for_deseq2(counts_fp: str, cohort_A_fp: str, cohort_B_fp: str) -> tuple[pd.DataFrame, pd.DataFrame]:
    """
    Preprocesses the counts matrix for use with DESeq2. Returns a tuple of the preprocessed counts matrix and the cohort data.
    """

    start_time = time()

    # Load dataset
    print("Loading dataset")
    counts = pd.read_csv(counts_fp, index_col=0)
    print(f"Dataset loaded in {round(time() - start_time, 2)} seconds")
    print("Dataset size:", counts.shape)

    # Filter out all-0 genes
    counts = counts.loc[:, counts.sum() != 0]

    # Read cohort data
    cohort_A = pd.read_csv(cohort_A_fp)
    cohort_B = pd.read_csv(cohort_B_fp)

    def determine_cohort(sample_id):
        if sample_id.lower() in cohort_A['COHORT A'].str.lower().values:
            return 'A'
        elif sample_id.lower() in cohort_B['COHORT B'].str.lower().values:
            return 'B'
        else:
            return 'Unknown'

    # Apply determine_cohort function to create 'cohort' column
    sids = counts.columns
    cohorts = [determine_cohort(sample_id) for sample_id in sids]

    # Print cohort sizes
    print('Cohort A size:', cohorts.count('A'))
    print('Cohort B size:', cohorts.count('B'))
    print('No Cohort size:', cohorts.count('Unknown'))

    # Drop Unknown cohort
    cohort_data = pd.DataFrame({'Sample': sids, 'Condition': cohorts})
    cohort_data = cohort_data[cohort_data['Condition'] != 'Unknown']

    # Set index to sample ID
    cohort_data = cohort_data.set_index('Sample')

    # Subset the counts to only include the columns that are in the cohorts
    counts_matrix = counts[cohort_data.index]
    print('Transposing data for deseq consumption')

    # NOTE: This transpose is not needed in the R version, because the R deseq expects the counts matrix to be in the format of genes as rows and samples as columns. 
    counts_matrix = counts_matrix.transpose()
    

    # Make sure rownames and colnames match
    print('Validating preprocessed data. Valid = ', all(counts_matrix.index == cohort_data.index))

    return counts_matrix, cohort_data


def deseq(counts_matrix: pd.DataFrame, metadata: pd.DataFrame) -> pd.DataFrame:

    print('Running Deseq')
    # run dispersion and log fold-change (LFC) estimation.
    dds = DeseqDataSet(counts=counts_matrix, metadata=metadata, design_factors="Condition")
    dds.deseq2()

    print('Running stat summary')
    # summary of statistical tests
    stat_res = DeseqStats(dds, n_cpus=8, contrast = ('Condition','A','B'))
    stat_res.summary()
    res = stat_res.results_df

    return res
    
start_time = time()

counts_matrix, cohort_data = preprocess_for_deseq2(
    counts_fp="./big_int.csv",
    cohort_A_fp="C:/Users/gglatzer/OneDrive - Fred Hutchinson Cancer Center/Documents/Oncoscape/Cohort_A.csv",
    cohort_B_fp="C:/Users/gglatzer/OneDrive - Fred Hutchinson Cancer Center/Documents/Oncoscape/Cohort_B.csv"
)

# counts_matrix, cohort_data = preprocess_for_deseq2(
#     counts_fp=r"C:\Users\gglatzer\GitHub\DifferentialExpression\count_table.csv",
#     cohort_A_fp=r"C:\Users\gglatzer\GitHub\DifferentialExpression\cha.csv",
#     cohort_B_fp=r"C:\Users\gglatzer\GitHub\DifferentialExpression\chb.csv"
# )

print('[PREPROCESSING FINISHED]. Runtime (s):', time() - start_time)

start_time_deseq = time()

deseq_summary = deseq(counts_matrix, cohort_data)

print('[DESEQ FINISHED]. Runtime (s):', time() - start_time_deseq)
print(deseq_summary)

print('[TOTAL RUNTIME (s)]:', time() - start_time)


Loading dataset
Dataset loaded in 1.48 seconds
Dataset size: (19979, 1298)
Cohort A size: 58
Cohort B size: 224
No Cohort size: 1016
Transposing data for deseq consumption
Validating preprocessed data. Valid =  True
[PREPROCESSING FINISHED]. Runtime (s): 1.8446526527404785
Running Deseq


Fitting size factors...
... done in 0.20 seconds.

Fitting dispersions...
... done in 35.22 seconds.

Fitting dispersion trend curve...
... done in 2.34 seconds.

c:\Users\gglatzer\AppData\Local\Programs\Python\Python312\Lib\site-packages\anndata\_core\views.py:144: RuntimeWarning: invalid value encountered in log
  results = super().__array_ufunc__(
Fitting MAP dispersions...
c:\Users\gglatzer\AppData\Local\Programs\Python\Python312\Lib\site-packages\deseqpyodide\utils.py:674: RuntimeWarning: invalid value encountered in log
  log_alpha_hat = np.log(alpha_hat)
c:\Users\gglatzer\AppData\Local\Programs\Python\Python312\Lib\site-packages\deseqpyodide\utils.py:715: RuntimeWarning: invalid value encountered in log
  x0=np.log(alpha_hat),
c:\Users\gglatzer\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\linalg\linalg.py:2120: RuntimeWarning: invalid value encountered in slogdet
  sign, logdet = _umath_linalg.slogdet(a, signature=signature)
... done in 155.81 seconds.

c:\Use

Running stat summary
Log2 fold change & Wald test p-value: Condition A vs B
[DESEQ FINISHED]. Runtime (s): 226.01384377479553
         baseMean  log2FoldChange     lfcSE      stat    pvalue      padj
A1BG     0.080212        0.100196  1.174031  0.085343  0.931989       NaN
A1CF     0.000000             NaN       NaN       NaN       NaN       NaN
A2M      6.083977        0.274073  0.079858  3.431990  0.000599  0.005458
A2ML1    0.013148       -0.130321  3.603477 -0.036165  0.971150       NaN
A3GALT2  0.003142       -0.029049  3.602803 -0.008063  0.993567       NaN
...           ...             ...       ...       ...       ...       ...
ZYG11A   0.012687       -0.130321  3.603477 -0.036165  0.971150       NaN
ZYG11B   2.325322        0.309209  0.127240  2.430115  0.015094  0.072132
ZYX      7.239711       -0.011824  0.077348 -0.152864  0.878505  0.952157
ZZEF1    2.660784        0.090997  0.124244  0.732406  0.463921  0.702139
ZZZ3     1.717435        0.250926  0.149711  1.676070  0.093

... done in 4.59 seconds.



### How will this scale?

In [12]:
ivan = pd.read_csv(r'C:\Users\gglatzer\GitHub\DifferentialExpression\count_table.csv', index_col=0)
ivan_size = ivan.shape[0] * ivan.shape[1]
print('Ivan\'s dataset sample size x gene count:', ivan_size)
ivan.shape

Ivan's dataset sample size x gene count: 485304


(60663, 8)

In [18]:
real = pd.read_csv(r"C:/Users/gglatzer/Downloads/1298_combatseq_log2tpm_sampleIDnew.csv")
a = pd.read_csv(r"C:/Users/gglatzer/OneDrive - Fred Hutchinson Cancer Center/Documents/Oncoscape/Cohort_A.csv")
b = pd.read_csv(r"C:/Users/gglatzer/OneDrive - Fred Hutchinson Cancer Center/Documents/Oncoscape/Cohort_B.csv")

a_cols = real.columns.str.lower().isin(a['COHORT A'].str.lower())
b_cols = real.columns.str.lower().isin(b['COHORT B'].str.lower())

real_shape_in_cohorts = real[real.columns[a_cols | b_cols]].shape
real_size = real_shape_in_cohorts[0] * real_shape_in_cohorts[1]

print('Brain UMAP dataset sample size x gene count:', real_size)
real_shape_in_cohorts

Brain UMAP dataset sample size x gene count: 5634078


(19979, 282)

In [19]:
times_larger = round(real_size / ivan_size, 2)
print(f'The Brain UMAP dataset is {times_larger} times larger than Ivan\'s dataset')

The Brain UMAP dataset is 11.61 times larger than Ivan's dataset


In [23]:
python_runtime = 113.66207528114319 # in seconds
R_runtime = 8.31981992721558 # in seconds

estimated_UMAP_python_runtime = python_runtime * times_larger
estimated_real_R_runtime = R_runtime * times_larger

print(f'Estimated Brain UMAP Python runtime {round(estimated_UMAP_python_runtime // 60)}m {round(estimated_UMAP_python_runtime % 60, 2)}s')
print(f'Estimated Brain UMAP R runtime (s): {round(estimated_real_R_runtime // 60)}m {round(estimated_real_R_runtime % 60, 2)}s')

Estimated Brain UMAP Python runtime 21m 59.62s
Estimated Brain UMAP R runtime (s): 1m 36.59s
